# 実行環境の作成

## 環境変数

In [3]:
RESAS_API_KEY = '	02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

# 都道府県一覧

In [1]:
import json
import urllib.request

def get_resas_preflist():
    url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    return res['result']

In [ ]:
preflist = get_resas_preflist()
print(preflist)

都道府県コードのリストを作成する場合

In [ ]:
pref_codes = [d.get('prefCode')
                 for d in get_resas_preflist()]
print(pref_codes)

# 市区町村一覧

In [17]:
import json
import urllib.request

def get_resas_citylist(prefCode=0,dc=None):
  
    # 都道府県'prefCode'を指定
    # 0の場合は全都道府県の市区町村
    if prefCode == 0:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    else:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}'.format(prefCode)
    
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())


    # 政令指定都市　の指定　True:政令指定都市統合　False：政令指定都市分割
    # Noneの場合は全市区町村
    if dc == True:
      return list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))
    elif dc == False:
      return list(filter(lambda x: x['bigCityFlag'] != '2', res['result']))
    else:
      return res['result']

In [ ]:
city1 = get_resas_citylist()
print(city1)

city2 = get_resas_citylist(dc=True)
print(city2)

city3 = get_resas_citylist(prefCode=28,dc=True)
print(city3)

city4 = get_resas_citylist(prefCode=28)
print(city4)